In [ ]:
import pandas as pd

In [ ]:
# Считываем csv-файл, отбирая только необходимые столбцы
# Заранее задаем типы данных в столбцах
cols = ['user_id','timestamp', 'answered_correctly']
dtypes = {'user_id': 'int32', 'timestamp': 'float32', 'answered_correctly': 'int32'}
df = pd.read_csv("./train.csv", usecols=cols, dtype=dtypes)

In [ ]:
# Рассчитываем столбец lag_time с предыдущим временем ответа
# Заменяем NaN для первых строк на 0
# Рассчитываем разницу во времени между каждым из ответов в новом столбце diff_time
# Переводим значения столбца diff_time в секунды
df['lag_time'] = df.groupby('user_id')['timestamp'].shift(1)
df['lag_time'] = df['lag_time'].fillna(0)
df['diff_time'] = df['timestamp']-df['lag_time']
df['diff_time'] = df['diff_time'].apply(lambda x: x/1000)

In [ ]:
# Рассчитываем среднее время ответа в группировке по ученикам и выводим топ-5
df_diff_time = df.where(df['diff_time']>0).groupby('user_id', as_index=False)[['diff_time', 'user_id']]
df_diff_time = df_diff_time.mean('diff_time')
df_diff_time['user_id'] = df_diff_time['user_id'].astype('int')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print('Топ-5 учеников по средней скорости ответов:')
    print(df_diff_time.sort_values(by='diff_time', ascending=True) \
                .rename(columns={'user_id': 'Ученик', 'diff_time': 'Среднее время ответа (сек)'}) \
                .head(5)[['Ученик','Среднее время ответа (сек)']] \
                .to_string(index=False)
          )

In [ ]:
# Рассчитываем количество правильных ответов в группировке по ученикам и выводим топ-5
df_corr_answ = df.where(df['answered_correctly']==1).groupby('user_id', as_index=False)[['user_id', 'answered_correctly']]
df_corr_answ = df_corr_answ.size()
df_corr_answ['user_id'] = df_corr_answ['user_id'].astype('int')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print('Топ-5 учеников по количеству правильных ответов:')
    print(df_corr_answ.sort_values(by='size', ascending=False)
          .rename(columns={'user_id': 'Ученик', 'size': 'Количество правильных ответов'}) \
          .head(5) \
          .to_string(index=False)
          )